In [290]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tqdm.notebook as tqdm
import copy
import os

from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.optim.lr_scheduler import StepLR


import sys
sys.path.append('/Users/Matt/projects/sgpvae/')
import sgpvae

## Data preparation.

In [291]:
transform=transforms.Compose([
        transforms.ToTensor(),
        ])

train_dataset = datasets.MNIST('../data', train=True, download=True,
                               transform=transform)
test_dataset = datasets.MNIST('../data', train=False,
                              transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

## Classifier training.

In [292]:
def train(model, train_loader, optimiser, epoch):
    model.train()
    batch_iter = tqdm.tqdm(enumerate(train_loader), desc='Batch')
    for batch_idx, (data, target) in batch_iter:
        optimiser.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimiser.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [293]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [140]:
cls = ClassificationNet()
cls_optimiser = optim.Adadelta(cls.parameters(), lr=1.0)
cls_scheduler = StepLR(optimiser, step_size=1, gamma=0.7)

In [141]:
for epoch in range(1, 14+1):
    train(cls, train_loader, cls_optimiser, epoch)
    test(cls, test_loader)
    cls_scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308736
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.715233
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.351968
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.841422
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.740839
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.491377
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.412979
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.549440
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.374979
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.254062
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.313764
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.480454
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.336323
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.432512
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.263767
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.231215
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.284241
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.168324
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.256327
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.153247
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.049946
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.081982
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.046863
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.056075



KeyboardInterrupt: 

## Set up EntroVAE and VAE datasets. 
* For ***EntroVAE*** use entropies as well as the data.
* For ***VAE***, dataset is simply the images.

In [258]:
pred_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)
probs = torch.zeros(len(train_dataset), 10)
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(pred_loader):
        probs[batch_idx*64:(batch_idx+1)*64] = cls(data).detach().exp()
        
entropies = - (probs * probs.log()).sum(1)

In [280]:
class EntroVAEDataset(torch.utils.data.Dataset):
    def __init__(self, data, entropy):
        self.data = data
        self.entropy = entropy
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        return self.data[idx, ...], self.entropy[idx]
    
entrovae_dataset = EntroVAEDataset(train_dataset.data, entropies)
    
entrovae_loader = torch.utils.data.DataLoader(entrovae_dataset, batch_size=64)
vae_loader = torch.utils.data.DataLoader(train_dataset.data, batch_size=64)

In [281]:
def train_vae(model, train_loader, optimiser, epoch):
    model.train()
    train_loss = 0
    batch_iter = tqdm.tqdm(enumerate(train_loader), desc='Batch')
    
    for batch_idx, data in batch_iter:
        optimiser.zero_grad()
        
        if isinstance(data, tuple) or isinstance(data, list):
            data, h = data
            data = data.view(-1, 784).float() / 255
            elbo = vae_elbo(model, data, h)
        else:
            data = data.view(-1, 784).float() / 255
            elbo = vae_elbo(model, data)
            
        loss = - elbo / data.shape[0]
        loss.backward()
        train_loss += loss.item()
        optimiser.step()
        
        if batch_idx % 10 == 0:
            batch_iter.set_postfix(loss=loss.item())
            
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [287]:
z_dim = 2
x_dim = 784

encoder = sgpvae.networks.LinearGaussian(x_dim, z_dim, [512, 256], min_sigma=1e-3)
decoder = sgpvae.networks.LinearNN(z_dim, x_dim, [256, 512])

vae = VAE(copy.deepcopy(encoder), copy.deepcopy(decoder), z_dim)
entrovae = EntroVAE(copy.deepcopy(encoder), copy.deepcopy(decoder), z_dim)

## VAE training.

In [227]:
vae_optimiser = optim.Adam(vae.parameters())
for epoch in range(1, 14+1):
    train_vae(vae, vae_loader, vae_optimiser, epoch)


====> Epoch: 1 Average loss: 2.7424



====> Epoch: 2 Average loss: 2.4491



====> Epoch: 3 Average loss: 2.3604



====> Epoch: 4 Average loss: 2.3146



====> Epoch: 5 Average loss: 2.2844


...


====> Epoch: 10 Average loss: 2.2161



====> Epoch: 11 Average loss: 2.2137



====> Epoch: 12 Average loss: 2.2082



====> Epoch: 13 Average loss: 2.2020



====> Epoch: 14 Average loss: 2.1981


In [232]:
with torch.no_grad():
    z = torch.zeros(100, 2)
    for i, x in enumerate(torch.linspace(-2, 2, 10, dtype=torch.float32)):
        for j, y in enumerate(torch.linspace(-2, 2, 10, dtype=torch.float32)):
            idx = 10 * i + j
            z[idx, :] = torch.tensor([x, y])
            
    sample = vae.decoder(z)
    
    filename = './samples/vae_sample'
    
    if os.path.exists(filename + '.png'):
        i = 1
        while os.path.exists(filename + '_' + str(i) + '.png'):
            i += 1
            
        filename = filename + '_' + str(i) + '.png'
        
    else:
        filename = filename + '.png'
    
    save_image(sample.view(100, 1, 28, 28), filename)

## EntroVAE training.

In [288]:
entrovae_optimiser = optim.Adam(entrovae.parameters())
for epoch in range(1, 14+1):
    train_vae(entrovae, entrovae_loader, entrovae_optimiser, epoch)


====> Epoch: 1 Average loss: 2.8167



====> Epoch: 2 Average loss: 2.5384



====> Epoch: 3 Average loss: 2.4656



====> Epoch: 4 Average loss: 2.4169



====> Epoch: 5 Average loss: 2.3847


...


====> Epoch: 10 Average loss: 2.3078



====> Epoch: 11 Average loss: 2.2985



====> Epoch: 12 Average loss: 2.2943



====> Epoch: 13 Average loss: 2.2823



====> Epoch: 14 Average loss: 2.2796


In [289]:
with torch.no_grad():
    z = torch.zeros(100, 2)
    for i, x in enumerate(torch.linspace(-2, 2, 10, dtype=torch.float32)):
        for j, y in enumerate(torch.linspace(-2, 2, 10, dtype=torch.float32)):
            idx = 10 * i + j
            z[idx, :] = torch.tensor([x, y])
            
    sample = entrovae.decoder(z)
    
    filename = './samples/entrovae_sample'
    
    if os.path.exists(filename + '.png'):
        i = 1
        while os.path.exists(filename + '_' + str(i) + '.png'):
            i += 1
            
        filename = filename + '_' + str(i) + '.png'
        
    else:
        filename = filename + '.png'
    
    save_image(sample.view(100, 1, 28, 28), filename)

## Compare disentalglement of latent space?
* Not sure what to use here tbh.

In [124]:
class ClassificationNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [286]:
class VAE(nn.Module):
    def __init__(self, encoder, decoder, latent_dim):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.latent_dim = latent_dim

    def latent_posterior(self, x):
        qz_mu, qz_sigma = self.encoder(x)

        # Reshape.
        qz_mu = qz_mu.transpose(0, 1)
        qz_sigma = qz_sigma.transpose(0, 1)

        return qz_mu, qz_sigma

    def sample_prior(self, num_samples=1):
        """Sample latent prior."""
        z = torch.randn(num_samples, self.latent_dim)

        return self.decoder(z)

    def sample_posterior(self, x, num_samples=1):
        """Sample latent posterior."""
        qz_mu, qz_sigma = self.latent_posterior(x)

        samples = qz_mu + qz_sigma * torch.randn(num_samples, *qz_mu.shape)

        # samples = [qz_mu + qz_sigma * torch.randn_like(qz_mu)
        #            for _ in range(num_samples)]

        return samples


class EntroVAE(VAE):
    def __init__(self, encoder, decoder, latent_dim, init_scale=1.):
        super().__init__(encoder, decoder, latent_dim)

        self.log_scale = nn.Parameter(torch.ones(latent_dim) * np.log(init_scale))

    def latent_posterior(self, x, h):
        qz_mu = self.encoder(x)[0]

        # TODO: there is no bloody way this is right.
        qz_sigma = h.unsqueeze(1).matmul(self.log_scale.exp().unsqueeze(0))
        
        # Add minimum sigma to prevent numerical instabilities.
        qz_sigma += 1e-3

        # Reshape.
        qz_mu = qz_mu.transpose(0, 1)
        qz_sigma = qz_sigma.transpose(0, 1)

        return qz_mu, qz_sigma

In [157]:
def loss_function(recon_x, x, mu, log_var):
    bce = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    kl = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    
    return bce + kl

In [265]:
def vae_elbo(model, x, h=None, num_samples=1):
    elbo = 0
    
    if h is None:
        qz_mu, qz_sigma = model.latent_posterior(x)
    else:
        qz_mu, qz_sigma = model.latent_posterior(x, h)

    for _ in range(num_samples):
        z = qz_mu + qz_sigma * torch.randn_like(qz_mu)
        
        # log p(x|z) term.
        recon_x = F.sigmoid(model.decoder(z.T))
        px_z_term = - F.binary_cross_entropy(recon_x, x, reduction='sum')
        elbo += px_z_term

    # Inner summation over samples from q(z).
    elbo /= num_samples

    # KL(q(z) | p(z))
    pf_mu = torch.zeros_like(qz_mu)
    pf_sigma = torch.ones_like(qz_sigma)
    kl_term = gaussian_diagonal_kl(qz_mu, qz_sigma.pow(2), pf_mu,
                                   pf_sigma.pow(2))
    kl_term = kl_term.sum()
    elbo += - kl_term

    return elbo

In [209]:
def gaussian_diagonal_kl(m1, v1, m2, v2):
    kl = 0.5 * ((v2 / v1).log() + (v1 + (m1 - m2) ** 2) / v2 - 1)

    # Sum over dimensions.
    kl = kl.sum(1)

    return kl